# Parallelization Lab

In this lab, you will be leveraging several concepts you have learned to obtain a list of links from a web page and crawl and index the pages referenced by those links - both sequentially and in parallel. Follow the steps below to complete the lab.

## Step 1: Use the requests library to retrieve the content from the URL below.

In [2]:
import requests

url = 'https://en.wikipedia.org/wiki/Data_science'

In [3]:
r = requests.get(url)
r.status_code
#r.content

200

## Step 2: Use BeautifulSoup to extract a list of all the unique links on the page.

In [4]:
from bs4 import BeautifulSoup as bs

In [5]:
#Extraer las etiquetas <a> con href
sopa = bs(r.text,'lxml')
links=[a for a in sopa.find_all('a', href=True)]
print('Todas las ligas encontradas en las etiquetas a:' ,len(links))
links = set(links)
links = list(links)
print('Ligas despùes de eliminar duplicados', len(links))

Todas las ligas encontradas en las etiquetas a: 470
Ligas despùes de eliminar duplicados 414


## Step 3: Use list comprehensions with conditions to clean the link list.

There are two types of links, absolute and relative. Absolute links have the full URL and begin with http while relative links begin with a forward slash (/) and point to an internal page within the wikipedia.org domain. Clean the respective types of URLs as follows.

- Absolute Links: Create a list of these and remove any that contain a percentage sign (%).
- Relativel Links: Create a list of these, add the domain to the link so that you have the full URL, and remove any that contain a percentage sign (%).
- Combine the list of absolute and relative links and ensure there are no duplicates.

In [6]:
domain = 'http://wikipedia.org'

In [7]:
#Absolute Links  - Extraer las etiquetas "a" con class "external text"
sopa = bs(r.text,'lxml')
links_abs=[a for a in sopa.find_all('a', {'class':'external text'})]
print('Links extraidos de la sopa:',len(links_abs))


Links extraidos de la sopa: 69


In [8]:
#Quitar etiquetas y href y title con su texto
for i in range(len(links_abs)):
    links_abs[i] = str(links_abs[i])
    links_abs[i] = links_abs[i].replace('<a class="external text" href="','')
    fin = links_abs[i].find('"')
    links_abs[i] = links_abs[i][:fin]
print('Totalidad de Links absolutos:',len(links_abs))
#Quirar las que tinen %
for i in range(len(links_abs)):
    if links_abs[i].find('%') >= 0:
        links_abs[i] = False

for i in range(links_abs.count(False)):
    links_abs.remove(False)

print('Totalidad de Links absolutos sin %:',len(links_abs))

links_abs = set(links_abs)
links_abs = list(links_abs)
print('Links absolutos únicos:',len(links_abs))    


Totalidad de Links absolutos: 69
Totalidad de Links absolutos sin %: 60
Links absolutos únicos: 58


In [9]:
#Relative - Links: Marcar como false todo lo que no empieze con <a href=
sopa = bs(r.text,'lxml')
links=[a for a in sopa.find_all('a', href=True)]
print('Links extraidos de la sopa:',len(links))

Links extraidos de la sopa: 470


In [10]:
#Marcar los que no empiezan con '<a href="/wiki/' y que tienen %

for i in range(len(links)):
    links[i]=str(links[i])
    if links[i].find('<a href="/wiki/') !=0 or links[i].find('%') >= 0:
        links[i]=False

#Remover de la lista los False
for i in range(links.count(False)):
    links.remove(False)
print('Despues de quitar los que no son ligas relativas y que tienen % quedan:',len(links))

Despues de quitar los que no son ligas relativas y que tienen % quedan: 205


In [11]:
#Quitar etiquetas y href y title con su texto
for i in range(len(links)):
    links[i]=links[i].replace('<a href="',domain)
    fin=links[i].find('"')
    links[i]=links[i][:fin]

In [12]:
print('Links relativos extraidos:',len(links))
links = set(links)
links = list(links)
print('Links relativos unicos:',len(links))

Links relativos extraidos: 205
Links relativos unicos: 157


In [13]:
for i in range(len(links_abs)):
    links.append(links_abs[i]) 
print('Total de links:',len(links))


Total de links: 215


In [14]:
links

['http://wikipedia.org/wiki/Independent_component_analysis',
 'http://wikipedia.org/wiki/Mathematics',
 'http://wikipedia.org/wiki/DeepDream',
 'http://wikipedia.org/wiki/NYU_Stern_Center_for_Business_and_Human_Rights',
 'http://wikipedia.org/wiki/Analytics',
 'http://wikipedia.org/wiki/Academic_publishing',
 'http://wikipedia.org/wiki/Anomaly_detection',
 'http://wikipedia.org/wiki/New_York_University',
 'http://wikipedia.org/wiki/University_of_Essex',
 'http://wikipedia.org/wiki/Logistic_regression',
 'http://wikipedia.org/wiki/Academy',
 'http://wikipedia.org/wiki/Harvard_Business_Review',
 'http://wikipedia.org/wiki/Open_science',
 'http://wikipedia.org/wiki/Data_science',
 'http://wikipedia.org/wiki/Journal_of_Machine_Learning_Research',
 'http://wikipedia.org/wiki/Conditional_random_field',
 'http://wikipedia.org/wiki/Computational_learning_theory',
 'http://wikipedia.org/wiki/Data_mining',
 'http://wikipedia.org/wiki/University_of_Michigan',
 'http://wikipedia.org/wiki/Empirical

## Step 4: Use the os library to create a folder called *wikipedia* and make that the current working directory.

In [15]:
import os

In [ ]:
os.mkdir('./wikipedia')

In [17]:
os.getcwd()

'/Users/al/Documents/ironhack/datamex0120/module-1/lab-parallelization/your-code'

In [18]:
os.chdir('./wikipedia')

In [19]:
os.getcwd()

'/Users/al/Documents/ironhack/datamex0120/module-1/lab-parallelization/your-code/wikipedia'

## Step 5: Write a function called index_page that accepts a link and does the following.

- Tries to request the content of the page referenced by that link.
- Slugifies the filename using the `slugify` function from the [python-slugify](https://pypi.org/project/python-slugify/) library and adds a .html file extension.
    - If you don't already have the python-slugify library installed, you can pip install it as follows: `$ pip install python-slugify`.
    - To import the slugify function, you would do the following: `from slugify import slugify`.
    - You can then slugify a link as follows `slugify(link)`.
- Creates a file in the wikipedia folder using the slugified filename and writes the contents of the page to the file.
- If an exception occurs during the process above, just `pass`.

In [20]:
from slugify import slugify

In [24]:
print(links[0])
print(slugify(links[0])+'.html')

http://wikipedia.org/wiki/Independent_component_analysis
http-wikipedia-org-wiki-independent-component-analysis.html


In [27]:
from tqdm import tqdm
def linkstofile(links,print_results=True):
    contex=0
    contok=0
    
    if type(links) == str:
        try:
            #crear archivo con slufied name
            f = open(slugify(links)+'.html','w+')
            #request
            r = requests.get(links)
            #escribir resultado del request en el archivo
            f.write(r.text)
            f.close()
            return
        except:
            if print_results == True:
                print("La página {} no pudo ser procesada".format(links))
            return
    
    for i in links:
        try:
            #crear archivo con slufied name
            f = open(slugify(i)+'.html','w+')
            #request
            r = requests.get(i)
            #escribir resultado del request en el archivo
            f.write(r.text)
            f.close()
            contok+=1
        except:
            contex+=1
    
    if print_results == True:
        print('extraidas:{}\nabortadas:{}'.format(contok,contex))

## Step 6: Sequentially loop through the list of links, running the index_page function each time.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [59]:
%%time 
for i in tqdm(range(len(links)),desc="Archivos procesados"):
    linkstofile(links[i],False)
    

Archivos procesados: 100%|██████████| 215/215 [02:54<00:00,  1.23it/s]

CPU times: user 27.5 s, sys: 1.4 s, total: 28.9 s
Wall time: 2min 54s


## Step 7: Perform the page indexing in parallel and note the difference in performance.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [47]:
import multiprocessing as mp

In [57]:
pool = mp.Pool(4)

La página //pubmed.ncbi.nlm.nih.gov/24436391 no pudo ser procesada
La página //pubmed.ncbi.nlm.nih.gov/19265007 no pudo ser procesada
La página //www.worldcat.org/issn/0960-3174 no pudo ser procesada
La página //www.worldcat.org/issn/0362-4331 no pudo ser procesada
La página //www.ncbi.nlm.nih.gov/pmc/articles/PMC4058759 no pudo ser procesada
La página //pubmed.ncbi.nlm.nih.gov/19535325 no pudo ser procesada
La página //www.worldcat.org/issn/0028-0836 no pudo ser procesada
La página //www.worldcat.org/issn/1465-4644 no pudo ser procesada
La página //www.worldcat.org/issn/0036-8075 no pudo ser procesada
La página //pubmed.ncbi.nlm.nih.gov/24482835 no pudo ser procesada


In [58]:
%%time
for i in tqdm(range(len(links)),desc='Archivos procesados'):
    pool.apply_async(linkstofile,args=(links[i],True))
    
pool.close()

Archivos procesados: 100%|██████████| 215/215 [00:00<00:00, 7065.16it/s]

CPU times: user 10.9 ms, sys: 4.64 ms, total: 15.6 ms
Wall time: 41.6 ms
